In [1]:
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
from sklearn.metrics import root_mean_squared_error, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import UNeuralNetwork as nn

In [2]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()

    output = model(x_batch.to('cpu'))  
    loss = loss_function(output, y_batch.to('cpu'))
    loss.backward()

    optimizer.step()
    return loss.cpu().item()  

def train_epoch(train_generator, model, loss_function, optimizer, callback=None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to('cpu'), batch_of_y.to('cpu'), optimizer, loss_function)

        if callback is not None:
            callback(model, batch_loss)

        epoch_loss += batch_loss * len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss / total

def trainer(count_of_epoch,
            batch_size,
            dataset,
            model,
            loss_function,
            optimizer,
            lr=0.001,
            callback=None):
    optima = optimizer(model.parameters(), lr=lr)
    for it in range(count_of_epoch):
        batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
        epoch_loss = train_epoch(train_generator=batch_generator,
                                 model=model,
                                 loss_function=loss_function,
                                 optimizer=optima,
                                 callback=callback)
        print(f'Эпоха {it+1}: ошибка = {epoch_loss}') 

def tester(model, dataset, batch_size):
    batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size)
    y_predict = []
    y_real = []
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to('cpu')
        y_batch = y_batch.to('cpu')

        output = model(x_batch)
        y_predict.extend(torch.argmax(output, dim=-1).cpu().numpy().tolist())
        y_real.extend(y_batch.cpu().numpy().tolist())
    print(classification_report(y_real, y_predict))    

In [3]:
class CNN(torch.nn.Module):
    @property
    def device(self):
        for param in self.parameters():
            return param.device

    def __init__(self):
        super(CNN, self).__init__()
        self.layers = torch.nn.Sequential()
        self.layers.add_module('conv1', torch.nn.Conv2d(1, 6, kernel_size=5))
        self.layers.add_module('relu1', torch.nn.ReLU())
        self.layers.add_module('pool1', torch.nn.MaxPool2d(kernel_size=2))
        self.layers.add_module('conv2', torch.nn.Conv2d(6, 16, kernel_size=5))
        self.layers.add_module('relu2', torch.nn.ReLU())
        self.layers.add_module('pool2', torch.nn.MaxPool2d(kernel_size=2))
        self.layers.add_module('flatten', torch.nn.Flatten())
        self.layers.add_module('linear1', torch.nn.Linear(16*4*4, 120))
        self.layers.add_module('relu3', torch.nn.ReLU())
        self.layers.add_module('linear2', torch.nn.Linear(120, 84))
        self.layers.add_module('relu4', torch.nn.ReLU())
        self.layers.add_module('linear3', torch.nn.Linear(84, 10))

    def forward(self, input):
        return self.layers(input)

In [8]:
#Загрузка данных
df = pd.read_csv('DATA/MNIST.csv').sample(2000, axis=0)
X = df.drop('label', axis=1)
y = df['label'].to_numpy(dtype=np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32).reshape(-1,1,28,28))
X_test = torch.from_numpy(X_test.astype(np.float32).reshape(-1,1,28,28))
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

data_train = torch.utils.data.TensorDataset(X_train, y_train)
data_test = torch.utils.data.TensorDataset(X_test, y_test)

In [9]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD

model = CNN().to('cpu')

In [10]:
#Обучение CNN
trainer(count_of_epoch=5,
        batch_size=64,
        dataset=data_train,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.1,
        callback=None) #callback - позволяет отслеживать обучение модели(логирование). Используется TensorBoard

Эпоха 1: ошибка = 2.254660213694853
Эпоха 2: ошибка = 1.7230839359059054
Эпоха 3: ошибка = 1.1147982070025275
Эпоха 4: ошибка = 0.5382336270107942
Эпоха 5: ошибка = 0.40615982090725616


In [11]:
tester(model=model,
       dataset=data_test,
       batch_size=64)

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        26
           1       0.94      1.00      0.97        31
           2       0.81      0.85      0.83        26
           3       0.86      0.83      0.84        23
           4       0.66      0.93      0.77        29
           5       0.87      0.92      0.89        36
           6       0.97      0.94      0.96        36
           7       0.79      0.92      0.85        36
           8       0.90      0.93      0.91        28
           9       0.91      0.34      0.50        29

    accuracy                           0.86       300
   macro avg       0.87      0.85      0.84       300
weighted avg       0.87      0.86      0.85       300



In [16]:
class UCNN:
    def __init__(self):
        self.layers = nn.USequential()
        self.layers.add_module('conv1', nn.UConv2d(1, 6, kernel_size=5))
        self.layers.add_module('relu1', nn.UReLU())
        self.layers.add_module('pool1', nn.UMaxPool2d(kernel_size=2))
        self.layers.add_module('conv2', nn.UConv2d(6, 16, kernel_size=5))
        self.layers.add_module('relu2', nn.UReLU())
        self.layers.add_module('pool2', nn.UMaxPool2d(kernel_size=2))
        self.layers.add_module('flatten', nn.UFlatten())
        self.layers.add_module('linear1', nn.ULinear(16*4*4, 120))
        self.layers.add_module('relu3', nn.UReLU())
        self.layers.add_module('linear2', nn.ULinear(120, 84))
        self.layers.add_module('relu4', nn.UReLU())
        self.layers.add_module('linear3', nn.ULinear(84, 10))

    def _train_epoch(self, batch_generator):
        for batch_of_x, batch_of_y in batch_generator:
            for X, y in zip(batch_of_x, batch_of_y):
                output = self.layers.forward(X)
                self.layers.backward(output, y, 'CEL')
                self.layers.step(0.001)
            print('Batch ready')
    
    def train(self, dataset, batch_size, count_of_epoch):
        for epoch_no in range(count_of_epoch):
            batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
            self._train_epoch(batch_generator)
            print(f'Эпоха {epoch_no+1} завершена')

    def test(self, dataset, batch_size):
        batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size)
        y_predict, y_real = [], []
        for x_batch, y_batch in batch_generator:
            for X, y in zip(batch_of_x, batch_of_y):
                output = self.layers.forward(X)
                y_predict.extend(torch.argmax(output, dim=-1).numpy().tolist())
                y_real.extend(y.numpy().tolist())
        print(classification_report(y_real, y_predict)) 

In [ ]:
uModel = UCNN()
uModel.train(data_train, 64, 5)
uModel.test(data_test, 64)